## AI CORONA MASK DETECTION

In [1]:
import cv2,os

data_path="C:/Users/SASIREKHA/desktop/dataset"
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

print(label_dict)
print(categories)
print(labels)



{'without_mask': 0, 'with_mask': 1}
['without_mask', 'with_mask']
[0, 1]


In [2]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            resized=cv2.resize(gray,(img_size,img_size))
            data.append(resized)
            target.append(label_dict[category])
            

        except Exception as e:
            print('Exception:',e)
            

In [3]:
import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)


In [4]:
from keras.utils import np_utils

In [5]:
new_target=np_utils.to_categorical(target)

In [6]:
np.save('data',data)
np.save('target',new_target)                                                       

# Convolutional Neural Network Architecture

In [9]:
import numpy as np

data=np.load('data.npy')
target=np.load('target.npy')

# Step 1: Specify the architecture

In [10]:
from keras.layers import Dense,Flatten,Activation,Conv2D,MaxPooling2D,Dropout
from keras.models import Sequential

In [11]:
model=Sequential()
model.add(Conv2D(200,(3,3),activation="relu",input_shape=data.shape[1:]))
model.add(Conv2D(60,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(100,(3,3),activation="relu"))
model.add(Conv2D(60,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation="relu"))
model.add(Dense(2,activation="softmax"))



# Step 2: Compile the Model

In [12]:
from tensorflow.keras.optimizers import Adam


In [13]:

model.compile(Adam(lr=0.001),loss="categorical_crossentropy",metrics=["accuracy"])

C:\Users\SASIREKHA\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [21]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [22]:
train_data.shape


(1238, 100, 100, 1)

In [23]:
test_data.shape

(138, 100, 100, 1)

# Step 3: Train the Model

In [24]:
from keras.callbacks import ModelCheckpoint

In [25]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)


Epoch 1/20
31/31 [==============================] - 97s 3s/step - loss: 0.6224 - accuracy: 0.6444 - val_loss: 0.5965 - val_accuracy: 0.6048
INFO:tensorflow:Assets written to: model-001.model\assets
Epoch 2/20
31/31 [==============================] - 89s 3s/step - loss: 0.4886 - accuracy: 0.7818 - val_loss: 0.3931 - val_accuracy: 0.8266
INFO:tensorflow:Assets written to: model-002.model\assets
Epoch 3/20
31/31 [==============================] - 84s 3s/step - loss: 0.2700 - accuracy: 0.8848 - val_loss: 0.1233 - val_accuracy: 0.9637
INFO:tensorflow:Assets written to: model-003.model\assets
Epoch 4/20
31/31 [==============================] - 83s 3s/step - loss: 0.1792 - accuracy: 0.9424 - val_loss: 0.1881 - val_accuracy: 0.9355
Epoch 5/20
31/31 [==============================] - 86s 3s/step - loss: 0.1359 - accuracy: 0.9525 - val_loss: 0.2278 - val_accuracy: 0.9315
Epoch 6/20
31/31 [==============================] - 86s 3s/step - loss: 0.0912 - accuracy: 0.9697 - val_loss: 0.1118 - val_acc

# saving the model

In [26]:
print(model.evaluate(test_data,test_target))

5/5 [==============================] - 3s 514ms/step - loss: 0.1494 - accuracy: 0.9638
[0.14940017461776733, 0.9637681245803833]


In [27]:
model_json=model.to_json()

In [28]:
f=open("C:/Users/SASIREKHA/CoronaMaskDectection.json","w")
f.write(model_json)
f.close()

In [29]:
model.save_weights("C:/Users/SASIREKHA/MaskdecWeights.h5")

In [30]:
import cv2
from keras.models import model_from_json

In [31]:
json_file=open("C:/Users/SASIREKHA/CoronaMaskDectection.json","r")
loaded_model_json=json_file.read()
loaded_model=model_from_json(loaded_model_json)
loaded_model.load_weights("C:/Users/SASIREKHA/MaskdecWeights.h5")

# Detecting Faces with and without masks

In [32]:

from keras.models import load_model
import cv2
import numpy as np




In [33]:
face_clsfr=cv2.CascadeClassifier('C:/Users/SASIREKHA/downloads/haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)


labels_dict={0:'MASK',1:'MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [34]:
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for x,y,w,h in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=loaded_model.predict(reshaped)
        label=np.argmax(result)

      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()
